In [1]:
import requests
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from random import randint
from time import sleep
from bs4 import BeautifulSoup
import numpy as np
import matplotlib.pyplot as plt
from sklearn import cluster, datasets
from sklearn.preprocessing import StandardScaler
from matplotlib.lines import Line2D
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from random import choice

In [2]:
secrets_file = open("secrets.txt","r")
string = secrets_file.read()
secrets_dict={}
for line in string.split('\n'):
    if len(line) > 0:
        secrets_dict[line.split(':')[0]]=line.split(':')[1]
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['cid'],
                                                           client_secret=secrets_dict['csecret']))

In [9]:
hot_100 = pd.read_csv('hot_100.csv')
master_playlist = pd.read_csv('master_playlist.csv')
final_master_playlist = pd.read_csv('final_master_playlist.csv')

In [7]:
def get_features_song(song_title):
    
    results = sp.search(q=song_title, limit=1)
    song_uri = results['tracks']['items'][0]["uri"]
    song_af = sp.audio_features(song_uri)
    song_af_df = pd.DataFrame(song_af)
    song_af_df = song_af_df.drop(['type','id','uri','track_href','analysis_url','duration_ms','time_signature'], axis=1)
    song_af_df = StandardScaler().fit_transform(song_af_df)
    
    return (song_af_df)

def get_cluster_song(song_title):
    
    song=get_features_song(song_title)
    cluster = kmeans.predict(song)
    
    return (cluster)

In [14]:
X = master_playlist.drop(['Artists','Song Title','Unnamed: 0'], axis=1)
X_prep = StandardScaler().fit_transform(X)

kmeans = KMeans(n_clusters=5, random_state=0).fit(X_prep)

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0.653,0.3340,9,-6.955,1,0.0313,0.411000,0.000035,0.1230,0.665,119.460
1,0.526,0.4360,1,-6.020,1,0.0232,0.098100,0.000008,0.1180,0.286,96.903
2,0.731,0.8370,9,-8.673,1,0.0477,0.000287,0.150000,0.0978,0.832,135.603
3,0.691,0.5070,11,-10.230,1,0.0343,0.627000,0.000000,0.4570,0.885,128.518
4,0.491,0.5830,7,-10.964,1,0.0376,0.185000,0.000000,0.4060,0.908,150.566
...,...,...,...,...,...,...,...,...,...,...,...
895,0.311,0.0264,0,-21.644,1,0.0370,0.731000,0.002050,0.1560,0.139,120.653
896,0.605,0.4550,5,-8.903,0,0.0301,0.595000,0.000000,0.0816,0.678,106.825
897,0.546,0.8180,0,-4.890,1,0.0310,0.048700,0.000000,0.0983,0.756,138.561
898,0.563,0.4410,4,-9.902,1,0.0437,0.773000,0.000000,0.3510,0.468,115.168


In [15]:
song_title, artist_name = input("Enter song title and artist name separated by a comma: ").lower().split(",")
print("song title: ", song_title)
print("artist name: ", artist_name.strip())
    
from random import choice
answer = 'y'
    
while answer == 'y':
    if (len(hot_100[hot_100['title'].str.contains(song_title)])>0):
        input_index = hot_100.index[hot_100['title'] == song_title].tolist()
        output_index = choice([i for i in range(0,9) if i not in [input_index]])
        print('Try listening to: ' + hot_100['title'].values[output_index] + ' by ' +  hot_100['artist'].values[output_index])
        answer = input("Do you want another suggestion? y/n ").lower()
    else:
        input_cluster = get_cluster_song(song_title)
        temp_df = final_master_playlist[final_master_playlist['cluster'] == input_cluster[0]]
        output_index = randint(0,len(temp_df))
        print('Try listening to: ' + temp_df['song'].values[output_index] + ' by ' +  temp_df['artists'].values[output_index])
        answer = input("Do you want another suggestion? y/n ").lower()
    

Enter song title and artist name separated by a comma: sound of silence, simon & darfunkle
song title:  sound of silence
artist name:  simon & darfunkle
Try listening to: chain of fools by aretha franklin
Do you want another suggestion? y/n n
